In [20]:
from pyteomics import fasta
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
import json
import os
import subprocess
import csv
from neo4j import GraphDatabase

In [10]:
#Data path
datadir = os.path.abspath("../../Data/")

#Datasets
OT_targets    = datadir+"/OpenTargets/targets" #OpenTargets | "targets" data
OT_directevi  = datadir+"/OpenTargets/associationByOverallDirect" #OpenTargets | Disease association - Direct evidence data
OT_indireevi  = datadir+"/OpenTargets/associationByOverallIndirect" #OpenTargets | Disease association - Indirect evidence data
diseasedb     = datadir+"/OpenTargets/Disease" #OpenTargets "Disease" intel

Uniprot_hsap  = datadir+"/Uniprotdb/human_proteome/UP000005640_9606.fasta" #Uniprot h. sapiens proteome
Unisprot             = datadir+"/Uniprotdb/uniprot_sprot_human.dat" #Uniprot core dataset, contains functional intel
Genemap       = datadir+"/Uniprotdb/HUMAN_9606_idmapping.dat" #Uniprot gene ID mapping

Orpha_en      = datadir+"/Disease_ontology/orpha_en_product1.json" #Orphanet rare disease xef

# Functions

## Data input

In [3]:
def genUniprot(unidata, genein):
    #Generate Uniprot <> Gene dictionary
    with open(genein) as f:
        read = f.readlines()
    unigene = {}
    for i in read:
        splitr = i.split("\t")
        if splitr[1] == "Gene_Name" or splitr[1] == "Gene_Synonym":
            unigene.setdefault(splitr[0], [])
            unigene[splitr[0]].append(splitr[2].strip("\n"))

    #Generate df of human proteome & dic of gene names to uniprot ids
    unidic    = {}
    fastain   = fasta.UniProt(unidata)
    count = 0
    for uni in fastain:
        count += 1
        unid = uni[0]['id']
        name = uni[0]['name']
        try:
            gene = unigene[unid][0]
        except:
            gene = uni[0]['gene_id']
        seq  = uni[1]
        unidic.setdefault(unid, {"Uniprot": unid, "Gene": gene, "Name": name, "Seq": seq})

    unidf = pd.DataFrame.from_dict(unidic, orient='index')

    return unidf

def parseOTtarget(data_dir, df):
    #Parse OpenTargets "targets" intel, return df
    #Add new intel cols to df
    intelcol = {"Ensembl_gene": 'id', \
                "Ensembl_trans": 'transcriptIds', \
                "Description": "functionDescriptions", \
                "Subcell_loc": "subcellularLocations", \
                "DB_entries": "dbXrefs", \
                "Bio_path": "pathways", \
                "Tx_approaches": "tractability"}
    for col in intelcol:
        df[col] = [list() for x in range(len(df.index))]
    data = []
    #Read in multi part .json to data list 
    for file in os.listdir(data_dir):
        path = data_dir + "/" + str(file).replace("._", "")
        with open(path, "r") as f:
            for line in f:
                data.append(json.loads(line))
                
    #Retrieve intel from protein coding entries, annotate df
    ensemtouni = {}
    for entry in data:
        try:
            entid = entry['proteinIds'][0]['id']
        except:
            entid = ""

        if entid in df.index:
            for j in intelcol:
                if intelcol[j] in entry and j != "Bio_path":
                    df.loc[entid, j].append(entry[intelcol[j]])
                    #Simplify biological pathways for easier querying
                elif intelcol[j] in entry and j == "Bio_path":
                    for path in entry[intelcol[j]]:
                        df.loc[entid, j].append([path['pathwayId'], path['pathway']])
                elif j == "Ensembl_gene":
                    ensemtouni.setdefault(entry[intelcol[j]], entid) 
    
    return df, ensemtouni

def parseOTdisease(dataloc):
    #Parse OT disease <> protein evidence
    data = {}
    for file in os.listdir(dataloc):
        if ".json" in str(file):
            with open(dataloc + "/" + str(file).replace("._", "")) as f:
                for line in f.readlines():
                    linedict = json.loads(line)
                    data.setdefault(linedict["diseaseId"], [linedict])
                    data[linedict["diseaseId"]].append(linedict)
    return data

def findDisease(database):
    #Parse high level OT disease intel
    data = {}
    count = 0
    for file in os.listdir(database):
        if ".json" in str(file):
            count += 1
            try:
                with open(database + "/" + str(file)) as f:
                    for line in f.readlines():
                        linedict = json.loads(line)
                        data.setdefault(linedict["id"], linedict)
            except:
                pass

    return data

def funcGen(unisprot):
    #Return a dictionary of Uniprot to function annotations
    outdic = {}
    with open(unisprot, "r") as f:
        infile = f.read().replace(";", " ")
        splitin= infile.split("//\n")
    #Iterate per entry, split block, retrieve Uniprot + individual functions
    for uni in splitin:
        count = 0
        allfunc = [line for line in uni.split("\n") if line.startswith('FT')]
        allac = [line for line in uni.split("\n") if line.startswith('AC')]
        try:
            uniprot = re.search(r"\bAC\s+([A-Za-z0-9]+)", uni).group().split()[1]
        except:
            uniprot = ""
        funcdic = {}
        func = {}
        curannot = ""
        #Iterate over functions, append to out dic
        for i in allfunc:
            match = re.match(r"^\S+(\s+)\S+", i)
            if len(match.group(1)) == 3:
                funcdic.setdefault(count, func)
                curannot = ""
                spliti = i.split()
                count += 1
                nums = re.findall(r'\d+', spliti[-1])
                nums = [int(num) for num in nums]
                func = {"type": spliti[1].replace(";", " "), "range": nums, "note": "", "evidence": "", "id": ""}
            else:
                if re.search(r"(?<=\/)\w+=+", i) != None:
                    search = re.search(r"(?<=\/)\w+=+", i)
                    curannot = search.group()[:-1]
                    func[curannot] = [i.split("=")[1].replace("\"", "")] 
                    spliti = i.split("/id=")
                else:
                    func[curannot] = func[curannot][0]+" ".join(i.split()[1:])
        outdic.setdefault(uniprot, funcdic)
    return outdic

def parseOrpha(datain):
    ##TO-DO
        #Alter output to df with cols = ontologies    
    #Read in Orphanet json, return dataframe of Orpha code / Name / other database crossreferences
    with open(datain) as f:
        dataj = json.load(f)
    outdic = {}
    for i in dataj["JDBOR"][0]["DisorderList"][0]["Disorder"]:
        orpha = i["OrphaCode"]
        nam   = i["Name"][0]["label"]
        xrefs = {}
        try:
            for j in i["ExternalReferenceList"][0]['ExternalReference']:
                xrefs.setdefault(j['Source'], {"ID": j['id'], "Mapping": j['DisorderMappingRelation'][0]["Name"][0]["label"]})
        except:
            pass
        outdic.setdefault(orpha, {"Name": nam, "Xref": xrefs})

    outdf = pd.DataFrame.from_dict(outdic)

    return outdf.transpose() 

def assocUniprotDisease(datadic, ensgenetouni, diseasexref):
    #Generate dictionary linking Uniprot ID to disease associations + scores
    uniprotdisease = {}
    for i in datadic:
        for ent in datadic[i]:
            try:
                Orpha   = int(ent['diseaseId'].split("_")[1])
                Score   = round(ent['score'], 3)
                Uniprot = ensgenetouni[ent['targetId']]
                Disnam  = diseasexref.loc[str(Orpha)]["Name"]
                uniprotdisease.setdefault(Uniprot, {Orpha: {"Disease": Disnam, "Score": Score}})
                uniprotdisease[Uniprot][Orpha] = {"Disease": Disnam, "Score": Score}
            except:
                pass
    return uniprotdisease

## Node / Edge generation

In [14]:
def genProtNode(df):
    #Generate Uniprot Protein node
    count = 0
    outcsv = [["id:ID", ":LABEL", "uniprot", "gene", "name", "seq", "ens_gene", "ens_trans", "description", "db_entries", "bio_path", "tx_approach"]]
    nodecodec = {}
    for index, row in df.iterrows():
        count += 1
        outcsv.append(["P"+str(count), "Protein", row["Uniprot"], row['Gene'], row['Name'], row['Seq'], row['Ensembl_gene'], row['Ensembl_trans'],\
                        row['Description'], row['DB_entries'], row['Bio_path'], row['Tx_approaches']])
        nodecodec.setdefault(index, "P"+str(count))
    return outcsv, nodecodec

def genFuncNode(funcdic):
    #Generate Uniprot Function node
    count = 0
    outcsv = [["id:ID", ":LABEL", "type",  "range", "alt_id", "evidence", "note"]]
    nodecodec = {}
    for uni in funcdic:
        for func in funcdic[uni]: 
            if len(funcdic[uni][func]) > 0:
                count += 1
                outcsv.append(["F"+str(count), "Function", funcdic[uni][func]["type"], funcdic[uni][func]["range"], funcdic[uni][func]["id"], funcdic[uni][func]["evidence"], funcdic[uni][func]["note"]])
                nodecodec.setdefault("F"+str(count), uni)
    return outcsv, nodecodec

def genDisNode(disdic, disedge):
    #Generate OpenTarget disease node
    count = 0
    outcsv = [["id:ID", ":LABEL", "id_ont", "name", "description"]]
    nodecodec = {}
    for dis in disdic:
        if dis in disedge:
            try:
                desc = disdic[dis]["description"]
            except:
                desc = ""
            count += 1
            outcsv.append(["D"+str(count), "Disease", dis, disdic[dis]["name"], desc.replace("\n", " ")])
            nodecodec.setdefault(dis, "D"+str(count))
    return outcsv, nodecodec

def genProtFuncEdge(funcod, protcod):
    #Generate Protein <> Functional edges
    outcsv = [[":START_ID",":END_ID", ":TYPE"]]
    count = 0
    for i in funcod:
        if funcod[i] in protcod:
            outcsv.append([i, protcod[funcod[i]], "is_feature"])

    return outcsv

def genDisProtEdge(disdic, protcod, discod, ensdic):
    #Generate Disease <> Functional edges
    outcsv = [[":START_ID",":END_ID", ":TYPE", "score"]]
    for i in disdic:
        for j in disdic[i]:
            if j["targetId"] in ensdic:
                outcsv.append([protcod[ensdic[j["targetId"]]], discod[i], "direct_evidence", float(j["score"])])
    return outcsv

## Graph read / write

In [5]:
def nodeToCSVprep(genenode, diseasenode):
    #Generate out lists for .csv out
    geneout = [["id:ID", ":LABEL", "name", "ensemble_id"]]
    for i in genenode:
        geneout.append([genenode[i]["ID"], "Gene", i, genenode[i]["Ensemble"]])
        
    disout = [["id:ID", ":LABEL", "name", "uri"]]
    for i in diseasenode:
        disout.append([diseasenode[i]["ID"], "Disease", i, diseasenode[i]["URI"]])
        
    return geneout, disout

def writecsv(infile, outfile):
    #Write out csv
    with open(outfile, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=";")
        writer.writerows(infile)

def run_query(query):
    #Submit query to graph database, retrieve output as df
    with driver.session() as session:
        result = session.run(query)
        rows = [record.data() for record in result]
        df = pd.DataFrame(rows)
        return df

# Main script

## Data prep

In [11]:
#Read in Uniprot proteome intel
start = time.time()
unidf = genUniprot(Uniprot_hsap, Genemap)
print("Time taken | Uniprot initial proteome = ", round(time.time() - start, 2))

#Read in OT target intel - DONE
start = time.time()
unidf, ensemtouni = parseOTtarget(OT_targets, unidf)
print("Time taken | OpenTargets targets intel = ", round(time.time() - start, 2))

#Lookup dic of ensemble gene ids to uniprot
start = time.time()
ensgenetouni = {}
genetouni    = {}
for index, row in unidf.iterrows():
    val = row["Ensembl_gene"]
   # print(val)
    for i in row["Ensembl_gene"]:
        ensgenetouni.setdefault(i.replace("\'", ""), index)
    genetouni.setdefault(row["Gene"], index)
print("Time taken | Ensemble <> Uniprot | Gene <> Uniprot lookup dic = ", round(time.time() - start, 2))

#Read in OT functional intel 
start = time.time()
funcdic = funcGen(Unisprot)
print("Time taken | Uniprot function intel = ", round(time.time() - start, 2))

#Read in OT disease intel
start = time.time()
diseasego = findDisease(diseasedb) 
print("Time taken | OpenTargets disease intel = ", round(time.time() - start, 2))

#Read in disease associations
start = time.time()
OT_direct = parseOTdisease(OT_directevi)
OT_indir  = parseOTdisease(OT_indireevi)
print("Time taken | OpenTargets disease intel = ", round(time.time() - start, 2))

#Generate dictionaries of Uniprot to disease assocations
start = time.time()
dirdic = assocUniprotDisease(OT_direct, ensgenetouni, diseasexref)
inddic = assocUniprotDisease(OT_indir, ensgenetouni, diseasexref)
print("Time taken | Uniprot disease association = ", round(time.time() - start, 2))

#Generate rare disease crossreference
start = time.time()
diseasexref = parseOrpha(Orpha_en)
print("Time taken | Rare disese xref = ", round(time.time() - start, 2))

Time taken | Uniprot initial proteome =  0.98
Time taken | OpenTargets targets intel =  3.86
Time taken | Ensemble <> Uniprot | Gene <> Uniprot lookup dic =  0.42
Time taken | Uniprot function intel =  5.8
Time taken | OpenTargets disease intel =  0.12
Time taken | Rare disese xref =  1.1
Time taken | OpenTargets disease intel =  13.46
Time taken | Uniprot disease association =  64.76


## Node / Edge generation

In [15]:
#Generate nodes / edges csv
protnodes, protcodec = genProtNode(unidf)
funcnodes, funccodec = genFuncNode(funcdic)
disnodes, discodec   = genDisNode(diseasego, OT_direct)
profuncedge          = genProtFuncEdge(funccodec, protcodec)
disprotedge          = genDisProtEdge(OT_direct, protcodec, discodec, ensgenetouni)
#transnod, transedge  = genscRNANodeEdge(scdf, genetouni, protcodec)

In [21]:
#Write out node + edge csv files
if os.path.exists("./Results") == False:
    os.mkdir("./Results")

#Write out nodes / edges csv
writecsv(protnodes, "./Results/protnodes.csv")
writecsv(disnodes, "./Results/disnodes.csv")
writecsv(funcnodes, "./Results/funcnodes.csv")
writecsv(profuncedge, "./Results/profuncedge.csv")
writecsv(disprotedge, "./Results/disprotedge.csv")

## Start graph

In [23]:
#Password
with open("./pass_ent.txt", "r") as f:
    password = f.read()

curdir = os.path.abspath("./Results/")
#Commands for stopping / generating / starting neo4j
cmd0 = ["sudo", "-S", "neo4j", "stop"]
cmd1 = [ \
        "sudo", "-S", "neo4j-admin", "database", "import", "full", "neo4j", \
        "--nodes="+ curdir+"/protnodes.csv", \
        "--nodes="+ curdir+"/disnodes.csv", \
        "--nodes="+ curdir+"/funcnodes.csv", \
        "--relationships="+ curdir+"/profuncedge.csv", \
        "--relationships="+ curdir+"/disprotedge.csv", \
        "--delimiter=;", \
        "--array-delimiter=|", \
        "--overwrite-destination", \
        "--verbose"
        ]
cmd2 = ["sudo", "-S", "neo4j", "start", "--verbose"]

#Stop server
result = subprocess.run(cmd0, input=password+"\n", capture_output=True, text=True)
if result.returncode == 0:
    print("Graph stopped")
else:
    print(f"Stop failed with error:\n{result.stderr}")

#Generate graph
result = subprocess.run(cmd1, input=password+"\n", capture_output=True, text=True)
if result.returncode == 0:
    print("Database import successful!")
else:
    print(f"Import failed with error:\n{result.stderr}")

#Start graph
result = subprocess.run(cmd2, input=password+"\n", capture_output=True, text=True)
if result.returncode == 0:
    print("Graph started")
else:
    print(f"Start failed with error:\n{result.stderr}")

Graph stopped
Database import successful!
Graph started


## Run cypher